In [43]:
import pandas as pd
from sklearn.decomposition import NMF
from utils.process import prepare_df, data_arrays
from sklearn import metrics
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
df = prepare_df()
df = df[df['Label'] != 'BENIGN']

NameError: name 'prepare_df' is not defined

In [ ]:
sample_size=10000

ranks_grid = np.unique([int(i) for i in np.exp(np.linspace(start=np.log(50), stop=np.log(1), num=15))])

results = []
for l1_ratio in (0, 0.1, 0.2, 0.5, 0.7, 1):
    for rank in ranks_grid:
        print(rank)
        if rank == 1:
            continue
        for trial in range(5):
            df_sampled = df.sample(n=sample_size)
            X, y = data_arrays(df_sampled)
            nmf = NMF(n_components=rank, init='random', max_iter=10000, l1_ratio=l1_ratio, beta_loss='kullback-leibler', solver='mu')
            W = nmf.fit_transform(X.T)
            H = nmf.components_

            ss = metrics.silhouette_score(X, np.argmax(H, axis=0))
            rmse = nmf.reconstruction_err_/np.linalg.norm(X)
            results.append([rank, trial, l1_ratio, metrics.rand_score(y, np.argmax(H, axis=0)), ss, rmse])

results = pd.DataFrame(results, columns=['rank', 'trial', 'l_1 ratio', 'score', 'silhouete score', 'rmse'])
results.to_csv('results_kl.csv')

In [ ]:
results = pd.read_csv('results_kl.csv')
palette = sns.color_palette("mako_r", 6)
sns.set_style("whitegrid")
ax = sns.lineplot(data=results, x='rank', y='silhouete score', hue='l_1 ratio', palette=palette)
ax.grid(axis='x')
ax.set(ylabel='Silhouete Score')
ax.set_ylim(0, 1)
ax.set_xlim(0, 50)
plt.savefig('/home/tosia/NASK/network_analyze/PLOTS/NNMF/silhouete_score_kl')

In [5]:
sns.lineplot(data=results, x='rank', y='score')

0.8081568411612696

In [50]:
results.groupby('rank')[ 'score', 'silhouete score', 'rmse'].mean()

/tmp/ipykernel_18844/3466361947.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  results.groupby('rank')[ 'score', 'silhouete score', 'rmse'].mean()


,score,silhouete score,rmse
rank,,,
2,0.637096,0.515721,1.369908
3,0.711885,0.585373,1.059201
4,0.744848,0.618570,0.874423
5,0.769496,0.638585,0.735315
7,0.791444,0.698668,0.453775
9,0.788868,0.652649,0.379542
12,0.795055,0.652366,0.232958
16,0.778503,0.421675,0.172706
21,0.770754,0.269444,0.131911


In [7]:
H

array([[1.06678903e-05, 3.42638582e-02, 2.31732471e-02, ...,
        4.73617586e-03, 1.62603968e-06, 0.00000000e+00],
       [0.00000000e+00, 7.14200215e-02, 2.85565158e-03, ...,
        9.99408837e-03, 0.00000000e+00, 0.00000000e+00],
       [3.91867382e-04, 3.67319483e-02, 3.94592329e-03, ...,
        2.65478844e-02, 5.95037386e-04, 2.12084600e-02],
       ...,
       [0.00000000e+00, 8.03240787e-02, 1.47509419e-02, ...,
        0.00000000e+00, 0.00000000e+00, 6.42545950e-04],
       [0.00000000e+00, 1.29196872e-01, 3.28424592e-02, ...,
        0.00000000e+00, 0.00000000e+00, 7.25826035e-01],
       [0.00000000e+00, 3.92825744e-02, 1.21713228e-01, ...,
        3.00721854e-01, 0.00000000e+00, 0.00000000e+00]])

In [8]:
np.argmax(H, axis=0)

array([ 7,  9,  6, ..., 11,  7,  5])

In [9]:
y

array(['PortScan', 'DoS GoldenEye', 'DoS Hulk', ..., 'DoS GoldenEye',
       'PortScan', 'DDoS'], dtype=object)